In [ ]:
# Member3 - MTCNN + DeepFace (single cell)

import cv2
import json
import numpy as np
import time
import logging
from dataclasses import dataclass
from datetime import datetime
from pathlib import Path
from typing import Optional, List, Dict, Any, Tuple

import hashlib
import pyttsx3
import threading
from queue import Queue, Empty


import customtkinter as ctk
from PIL import Image, ImageTk
from tkinter import messagebox, Toplevel
from pyzbar.pyzbar import decode
from deepface import DeepFace
from mtcnn import MTCNN

# ------------------------
# Result Logger
# ------------------------
class ResultLogger:
    def __init__(self, results_directory: str = "results"):
        self.results_dir = Path(results_directory)
        self.results_dir.mkdir(exist_ok=True)
        
        # Single file to store all results
        self.results_file = self.results_dir / "all_results.json"
        
    def save_result(self, 
                   member_name: str,
                   student_id: str, 
                   score: float,
                   runtime: float,
                   success: bool,
                   timestamp: Optional[str] = None) -> bool:
        """
        Save verification result by appending to accumulated results.
        """
        try:
            if timestamp is None:
                timestamp = datetime.now().isoformat()
            
            # Create result data with additional metadata
            result_data = {
                "member": member_name,
                "student_id": student_id,
                "score": round(score, 4),
                "runtime": round(runtime, 4),
                "success": success,  # ADD THIS FIELD
                "timestamp": timestamp,
                "test_run": self._get_next_run_number(member_name)
            }
            
            # Load existing results or create new list
            results = self._load_all_results()
            results.append(result_data)
            
            # Save back to file
            with open(self.results_file, 'w', encoding='utf-8') as f:
                json.dump(results, f, indent=2, ensure_ascii=False)
            
            status = "SUCCESS" if success else "FAILED"
            print(f"✓ Result saved for {member_name}: Run #{result_data['test_run']}, Score={score:.3f}, Runtime={runtime:.3f}s, Status={status}")
            return True
            
        except Exception as e:
            print(f"✗ Failed to save result for {member_name}: {e}")
            return False
    
    def _load_all_results(self) -> List[Dict]:
        """Load all existing results"""
        if self.results_file.exists():
            try:
                with open(self.results_file, 'r', encoding='utf-8') as f:
                    return json.load(f)
            except (json.JSONDecodeError, FileNotFoundError):
                return []
        return []
    
    def _get_next_run_number(self, member_name: str) -> int:
        """Get the next run number for a specific member"""
        results = self._load_all_results()
        member_results = [r for r in results if r.get('member') == member_name]
        return len(member_results) + 1
    
result_logger = ResultLogger()

# ------------------------
# Config & Logging
# ------------------------
@dataclass
class Config:
    FACE_DB_PATH: str = "student_faces"
    LOG_FILE: str = "verification_log.json"
    STUDENT_DATA_FILE: str = "student_database.json"
    SUPPORTED_EXTENSIONS: List[str] = None

    # Camera
    CAMERA_WIDTH: int = 640
    CAMERA_HEIGHT: int = 480
    CAMERA_FPS: int = 25

    # Verification thresholds
    MATCH_THRESHOLD: float = 0.75
    REQUIRED_CONSECUTIVE: int = 3

    # Quality thresholds
    MIN_FACE_SIZE: int = 80
    PREFERRED_FACE_SIZE: int = 120

    # Timeouts
    VERIFICATION_TIMEOUT: int = 30
    QR_SCAN_TIMEOUT: int = 15

    def __post_init__(self):
        if self.SUPPORTED_EXTENSIONS is None:
            self.SUPPORTED_EXTENSIONS = ['.jpg', '.jpeg', '.png', '.bmp']

logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[logging.FileHandler('system.log'), logging.StreamHandler()]
)
logger = logging.getLogger(__name__)

# ------------------------
# Backend
# ------------------------
class StudentSystem:
    def __init__(self, config: Config = None):
        self.config = config or Config()
        self._setup_directories()
        self.load_student_data()
        logger.info("Student Verification System backend initialized")

    def _setup_directories(self):
        Path(self.config.FACE_DB_PATH).mkdir(exist_ok=True)

    def load_student_data(self):
        try:
            if Path(self.config.STUDENT_DATA_FILE).exists():
                with open(self.config.STUDENT_DATA_FILE, 'r', encoding='utf-8') as f:
                    self.student_data = json.load(f)
                logger.info(f"Loaded data for {len(self.student_data)} students")
            else:
                self.student_data = {}
                logger.info("No existing student data found, starting fresh")
        except (json.JSONDecodeError, IOError) as e:
            logger.error(f"Error loading student data: {e}")
            self.student_data = {}

    def get_student_info(self, student_id: str) -> Optional[Dict[str, Any]]:
        return self.student_data.get(student_id)

    def load_registered_faces(self, student_id: str) -> List[np.ndarray]:
        folder = Path(self.config.FACE_DB_PATH) / student_id
        if not folder.is_dir():
            logger.info(f"No face folder found for student {student_id}")
            return []
        images = []
        for file_path in folder.glob("*"):
            if file_path.suffix.lower() in self.config.SUPPORTED_EXTENSIONS:
                try:
                    img = cv2.imread(str(file_path))
                    if img is not None and img.size > 0:
                        images.append(img)
                    else:
                        logger.warning(f"Image file is empty or unreadable: {file_path}")
                except Exception as e:
                    logger.warning(f"Failed to load image {file_path}: {e}")
        logger.info(f"Total images loaded for student {student_id}: {len(images)}")
        return images

    def log_verification(self, student_id: str, name: str, program: str, success: bool,
                         timestamp: str, score: float = 0.0, verification_time: float = 0.0):
        log_entry = {
            "student_id": student_id, 
            "name": name, 
            "program": program, 
            "success": success,
            "timestamp": timestamp, 
            "score": round(float(score), 6), 
            "verification_time": round(float(verification_time), 6),
            "session_id": hashlib.md5(f"{timestamp}{student_id}".encode()).hexdigest()[:8]
        }
        
        try:
            # Load existing logs with proper error handling
            logs = []
            if Path(self.config.LOG_FILE).exists():
                try:
                    with open(self.config.LOG_FILE, 'r', encoding='utf-8') as f:
                        content = f.read().strip()
                        if content:
                            logs = json.loads(content)
                        else:
                            logs = []
                except (json.JSONDecodeError, ValueError) as e:
                    logger.error(f"Corrupted log file detected, backing up and starting fresh: {e}")
                    # Backup corrupted file
                    backup_name = f"{self.config.LOG_FILE}.backup_{int(time.time())}"
                    Path(self.config.LOG_FILE).rename(backup_name)
                    logs = []
            
            # Ensure logs is a list
            if not isinstance(logs, list):
                logger.warning("Log file contains invalid format, starting fresh")
                logs = []
            
            logs.append(log_entry)
            
            # Keep only last 1000 logs
            if len(logs) > 1000:
                logs = logs[-1000:]
            
            # Write with atomic operation using temporary file
            temp_file = f"{self.config.LOG_FILE}.tmp"
            with open(temp_file, 'w', encoding='utf-8') as f:
                json.dump(logs, f, indent=2, ensure_ascii=False)
            
            # Atomic move (rename) to replace the original file
            Path(temp_file).replace(self.config.LOG_FILE)
                
            status = "SUCCESS" if success else "FAILED"
            logger.info(f"Verification logged: {name} - {status} (Score: {score:.3f})")
            
        except Exception as e:
            logger.error(f"Error logging verification: {e}")
            # Try to log to backup location
            try:
                backup_log = f"verification_log_backup_{int(time.time())}.json"
                with open(backup_log, 'w', encoding='utf-8') as f:
                    json.dump([log_entry], f, indent=2, ensure_ascii=False)
                logger.info(f"Logged to backup file: {backup_log}")
            except Exception as backup_error:
                logger.error(f"Failed to create backup log: {backup_error}")

    def view_logs(self) -> str:
        """Return formatted logs for display"""
        try:
            if not Path(self.config.LOG_FILE).exists():
                return "No verification logs found."
            
            with open(self.config.LOG_FILE, 'r', encoding='utf-8') as f:
                logs = json.load(f)
            
            if not logs:
                return "No verification logs found."
            
            # Show last 20 logs
            recent_logs = logs[-20:]
            output = "RECENT VERIFICATION LOGS\n" + "="*50 + "\n\n"
            
            for log in reversed(recent_logs):
                status = "✓ SUCCESS" if log['success'] else "✗ FAILED"
                timestamp = datetime.fromisoformat(log['timestamp']).strftime("%Y-%m-%d %H:%M:%S")
                score = log.get('score', 0) * 100
                
                output += f"{timestamp} | {status} | {log['name']} ({log['student_id']}) | Score: {score:.1f}%\n"
            
            return output
            
        except Exception as e:
            return f"Error loading logs: {e}"

    def export_verification_report(self) -> str:
        """Export verification report"""
        try:
            if not Path(self.config.LOG_FILE).exists():
                return "No verification logs found to export."
            
            with open(self.config.LOG_FILE, 'r', encoding='utf-8') as f:
                logs = json.load(f)
            
            if not logs:
                return "No verification data to export."
            
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            report_file = f"verification_report_{timestamp}.json"
            
            total = len(logs)
            successful = sum(1 for log in logs if log['success'])
            success_rate = (successful / total * 100) if total > 0 else 0
            
            report = {
                'generated_at': datetime.now().isoformat(),
                'summary': {
                    'total_attempts': total,
                    'successful_verifications': successful,
                    'success_rate_percentage': round(success_rate, 2),
                },
                'recent_logs': logs[-50:]
            }
            
            with open(report_file, 'w', encoding='utf-8') as f:
                json.dump(report, f, indent=2, ensure_ascii=False)
                
            return f"Report successfully exported to {report_file}"
            
        except Exception as e:
            return f"Failed to generate report: {e}"

class RobustTTS:
    def __init__(self):
        self.speech_queue = Queue()
        self.worker_thread = threading.Thread(target=self._worker_loop, daemon=True)
        self.worker_thread.start()

    def _worker_loop(self):
        while True:
            try:
                text = self.speech_queue.get()
                if text is None:
                    break
                
                engine = pyttsx3.init(driverName='sapi5')
                voices = engine.getProperty('voices')
                if voices:
                    for voice in voices:
                        if 'female' in voice.name.lower() or 'zira' in voice.name.lower():
                            engine.setProperty('voice', voice.id)
                            break
                engine.setProperty('rate', 155)
                engine.setProperty('volume', 0.9)
                
                engine.say(text)
                engine.runAndWait()
                del engine
                
            except Exception as e:
                logging.error(f"TTS error: {e}")

    def speak(self, text: str):
        if text and text.strip():
            while not self.speech_queue.empty():
                try:
                    self.speech_queue.get_nowait()
                except Empty:
                    break
            self.speech_queue.put(text.strip())

    def cleanup(self):
        self.speech_queue.put(None)
        self.worker_thread.join(timeout=2.0)


# ------------------------
# UI
# ------------------------
class App(ctk.CTk):
    def __init__(self, student_system: StudentSystem):
        super().__init__()
        self.system = student_system
        self.member_name = "Member3"
        self.tts_system = RobustTTS()
        self.mtcnn = MTCNN()

        # State initialization
        self.cap = None
        self.verification_active = False
        self.current_student_id = None
        self.current_student_name = None
        self.current_student_program = None
        self.consecutive_matches = 0
        self.best_score = 0.0
        self.frame_count = 0
        self.verification_step = "idle"
        self.verification_start_time = None
        self.qr_scan_start_time = None
        self.max_verification_time = 30
        self.max_qr_scan_time = 15
        self.qr_detected = False

        # DeepFace settings
        self.deepface_model = "VGG-Face"
        self.ref_embeddings = []
        self.PROCESS_EVERY_N_FRAMES = 5

        # UI setup
        self.setup_ui()

        self.ctk_img = None  # Add a member variable to hold the reference

    def setup_ui(self):
        # Window configuration
        self.title("🎓 Student Verification System")
        self.geometry("1100x720")  # Standardize window size
        ctk.set_appearance_mode("Dark")  # Dark theme for consistency
        ctk.set_default_color_theme("blue")  # Unified color theme

        # --- Layout ---
        self.grid_columnconfigure(1, weight=1)
        self.grid_rowconfigure(0, weight=1)

        # Sidebar Frame (consistent size and alignment)
        self.sidebar_frame = ctk.CTkFrame(self, width=200, corner_radius=0)
        self.sidebar_frame.grid(row=0, column=0, rowspan=4, sticky="nsew")
        self.sidebar_frame.grid_rowconfigure(5, weight=1)

        self.logo_label = ctk.CTkLabel(self.sidebar_frame, text="Main Menu", font=ctk.CTkFont(size=20, weight="bold"))
        self.logo_label.grid(row=0, column=0, padx=20, pady=(20, 10))

        self.verify_button = ctk.CTkButton(self.sidebar_frame, text="Verify Student", command=self.verify_student_frame_event)
        self.verify_button.grid(row=1, column=0, padx=20, pady=10)

        self.logs_button = ctk.CTkButton(self.sidebar_frame, text="View Logs", command=self.view_logs_frame_event)
        self.logs_button.grid(row=2, column=0, padx=20, pady=10)

        self.export_button = ctk.CTkButton(self.sidebar_frame, text="Export Report", command=self.export_report_event)
        self.export_button.grid(row=3, column=0, padx=20, pady=10)

        self.exit_button = ctk.CTkButton(self.sidebar_frame, text="Exit", fg_color="transparent", border_width=2, text_color=("gray10", "#DCE4EE"), command=self.on_closing)
        self.exit_button.grid(row=6, column=0, padx=20, pady=20, sticky="s")

        # Verification Frame (standardized layout)
        self.verify_frame = ctk.CTkFrame(self, corner_radius=0, fg_color="transparent")
        self.verify_frame.grid_columnconfigure(0, weight=1)

        self.camera_label = ctk.CTkLabel(self.verify_frame, text="")
        self.camera_label.grid(row=0, column=0, padx=10, pady=10)

        self.status_label = ctk.CTkLabel(self.verify_frame, text="Welcome! Click 'Start Verification' to begin.", font=ctk.CTkFont(size=16))
        self.status_label.grid(row=1, column=0, padx=10, pady=5)

        self.info_label = ctk.CTkLabel(self.verify_frame, text="", font=ctk.CTkFont(size=14))
        self.info_label.grid(row=2, column=0, padx=10, pady=5)

        self.progress_bar = ctk.CTkProgressBar(self.verify_frame, width=300)
        self.progress_bar.set(0)
        self.progress_bar.grid(row=3, column=0, padx=10, pady=5)

        self.start_button = ctk.CTkButton(self.verify_frame, text="Start Verification", command=self.start_verification_process)
        self.start_button.grid(row=4, column=0, padx=10, pady=10)

        # Logs Frame (consistent with other members)
        self.logs_frame = ctk.CTkFrame(self, corner_radius=0, fg_color="transparent")
        self.logs_frame.grid_columnconfigure(0, weight=1)
        self.logs_frame.grid_rowconfigure(0, weight=1)
        self.log_textbox = ctk.CTkTextbox(self.logs_frame, width=700, font=("Courier New", 12))
        self.log_textbox.grid(row=0, column=0, padx=20, pady=20, sticky="nsew")

        # Initial frame selection
        self.select_frame_by_name("verify")
        self.protocol("WM_DELETE_WINDOW", self.on_closing)

    def select_frame_by_name(self, name):
        self.verify_frame.grid_remove()
        self.logs_frame.grid_remove()

        if name == "verify":
            self.verify_frame.grid(row=0, column=1, sticky="nsew")
        elif name == "logs":
            self.logs_frame.grid(row=0, column=1, sticky="nsew")

    def verify_student_frame_event(self):
        self.select_frame_by_name("verify")

    def view_logs_frame_event(self):
        self.select_frame_by_name("logs")
        logs = self.system.view_logs()
        self.log_textbox.configure(state="normal")
        self.log_textbox.delete("1.0", "end")
        self.log_textbox.insert("1.0", logs)
        self.log_textbox.configure(state="disabled")

    def export_report_event(self):
        message = self.system.export_verification_report()
        messagebox.showinfo("Export Report", message)

    def check_timeouts(self):
        current_time = time.time()
        if self.verification_step == "qr_scan" and self.qr_scan_start_time:
            if current_time - self.qr_scan_start_time > self.system.config.QR_SCAN_TIMEOUT:
                self.handle_timeout("qr_scan")
                return True
        elif self.verification_step == "face_verify" and self.verification_start_time:
            if current_time - self.verification_start_time > self.system.config.VERIFICATION_TIMEOUT:
                self.handle_timeout("face_verify")
                return True
        return False

    def handle_timeout(self, t):
        if t == "qr_scan":
            messagebox.showwarning("Timeout", "QR code scanning timed out. Please try again.")
            self.reset_labels()
        elif t == "face_verify":
            messagebox.showwarning("Timeout", "Face verification timed out. Please try again.")
            self.reset_labels()
        if hasattr(self, 'current_student_id') and self.current_student_id:
            timestamp = datetime.now().isoformat()
            self.system.log_verification(
                self.current_student_id or "UNKNOWN",
                self.current_student_name or "Unknown User",
                self.current_student_program or "Unknown Program",
                False,
                timestamp,
                score=self.best_score,
                verification_time=time.time() - self.verification_start_time if self.verification_start_time else 0
            )
        self.finalize_verification(False)

    def start_verification_process(self):
        if self.verification_active:
            return
        self.cap = cv2.VideoCapture(0)
        if not self.cap.isOpened():
            messagebox.showerror("Camera Error", "Cannot open camera.")
            return
        self.cap.set(cv2.CAP_PROP_FRAME_WIDTH, self.system.config.CAMERA_WIDTH)
        self.cap.set(cv2.CAP_PROP_FRAME_HEIGHT, self.system.config.CAMERA_HEIGHT)
        self.verification_active = True
        self.verification_step = "qr_scan"
        self.start_button.configure(state="disabled", text="Verification in Progress...")
        self.verification_start_time = time.time()
        self.qr_scan_start_time = time.time()
        self.qr_detected = False
        self.update_frame()

    def stop_verification_process(self):
        self.verification_active = False
        self.verification_step = "idle"
        if self.cap:
            self.cap.release()
            self.cap = None
        self.start_button.configure(state="normal", text="Start Verification")
        self.current_student_id = None
        self.consecutive_matches = 0
        self.best_score = 0.0
        self.frame_count = 0
        self.progress_bar.set(0)
        self.qr_detected = False

    def update_frame(self):
        if not self.verification_active: return
        if self.check_timeouts(): return 
        ret, frame = self.cap.read()
        if not ret:
            self.after(33, self.update_frame)
            return
        frame = cv2.flip(frame, 1)
        self.frame_count += 1
        if self.verification_step == "qr_scan":
            self.handle_qr_scan(frame)
        elif self.verification_step == "face_verify":
            self.handle_face_verify(frame)
        img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        img = Image.fromarray(img)
        self.ctk_img = ctk.CTkImage(light_image=img, dark_image=img, size=(self.system.config.CAMERA_WIDTH, self.system.config.CAMERA_HEIGHT))
        self.camera_label.configure(image=self.ctk_img)
        self.camera_label.image = self.ctk_img
        self.after(100, self.update_frame)

    def preprocess_qr(self, frame):
        """
        Enhance QR code region for better detection under various lighting.
        """
        # Convert to grayscale
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        # Histogram equalization for contrast
        eq = cv2.equalizeHist(gray)
        # Adaptive thresholding for binarization
        thresh = cv2.adaptiveThreshold(eq, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                                       cv2.THRESH_BINARY, 11, 2)
        # Merge back to 3 channels for compatibility
        processed = cv2.merge([thresh, thresh, thresh])
        return processed

    def handle_qr_scan(self, frame):
        self.status_label.configure(text="Step 1: Scan Student QR Code")
        self.info_label.configure(text="Position the QR code in the camera view.")
        # Preprocess the QR code frame
        qr_frame = self.preprocess_qr(frame)
        decoded_objects = decode(qr_frame)
        for obj in decoded_objects:
            qr_data = obj.data.decode("utf-8")
            student = self.system.get_student_info(qr_data)
            if student and not self.qr_detected:
                self.qr_detected = True
                self.current_student_id = qr_data
                self.current_student_name = student.get("name")
                self.current_student_program = student.get("program")
                points = obj.polygon
                hull = cv2.convexHull(np.array([point for point in points], dtype=np.float32))
                cv2.polylines(frame, [np.int32(hull)], True, (0, 255, 0), 3)
                self.status_label.configure(text="QR Code Detected! Preparing verification...")
                self.info_label.configure(text=f"Student: {self.current_student_name} - Loading model...")
                self.after(300, self.prepare_face_verification)
                return

    def prepare_face_verification(self):
        if not self.current_student_id:
            return

        self.status_label.configure(text="Preparing face verification...", text_color="yellow")
        self.info_label.configure(text=f"Student: {self.current_student_name} ({self.current_student_program})")

        self.ref_embeddings = []
        registered_faces = self.system.load_registered_faces(self.current_student_id)

        loaded = 0
        for ref_img in registered_faces[:20]:
            try:
                proc = self.preprocess_face(ref_img)

                rep = DeepFace.represent(
                    img_path=proc,
                    model_name=self.deepface_model,
                    detector_backend="mtcnn",
                    enforce_detection=False
                )

                if isinstance(rep, list) and len(rep) > 0 and 'embedding' in rep[0]:
                    emb = np.array(rep[0]['embedding'], dtype=np.float32)
                    norm = np.linalg.norm(emb)
                    if norm > 0:
                        emb = emb / norm
                        self.ref_embeddings.append(emb)
                        loaded += 1
                        logger.info(f"Loaded ref embedding norm: {norm:.3f}")
                    else:
                        logger.warning("Reference embedding has zero norm")
            except Exception as e:
                logger.warning(f"Ref embedding error: {e}")
                continue

        logger.info(f"Loaded {loaded} reference embeddings for {self.current_student_id}")
        if loaded == 0:
            self.info_label.configure(text="No valid reference embeddings loaded. Check face DB.", text_color="red")

        self.verification_step = "face_verify"
        self.verification_start_time = time.time()


    def preprocess_face(self, face_img):
        """
        Enhance face image for better recognition under various lighting.
        """
        if face_img is None or face_img.size == 0:
            return face_img
        # Convert BGR -> LAB for CLAHE on brightness
        lab = cv2.cvtColor(face_img, cv2.COLOR_BGR2LAB)
        l, a, b = cv2.split(lab)
        clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(4, 4))
        cl = clahe.apply(l)
        limg = cv2.merge((cl, a, b))
        enhanced = cv2.cvtColor(limg, cv2.COLOR_LAB2BGR)

        # Convert to RGB (DeepFace expects RGB)
        rgb = cv2.cvtColor(enhanced, cv2.COLOR_BGR2RGB)

        # Resize to DeepFace expected input size
        resized = cv2.resize(rgb, (224, 224), interpolation=cv2.INTER_LINEAR)
        return resized

    def handle_face_verify(self, frame):
        self.status_label.configure(text="Step 2: Position Your Face Clearly")
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        try:
            detections = self.mtcnn.detect_faces(rgb_frame)
        except Exception as e:
            logger.warning(f"MTCNN detection error: {e}")
            detections = []

        if detections:
            best = max(detections, key=lambda f: f['box'][2] * f['box'][3])
            x, y, w, h = best['box']
            x, y = max(0, x), max(0, y)
            w = min(w, frame.shape[1] - x)
            h = min(h, frame.shape[0] - y)

            face_img = frame[y:y+h, x:x+w]
            processed_face = self.preprocess_face(face_img)

            final_score = 0.0
            if self.frame_count % self.PROCESS_EVERY_N_FRAMES == 0 and self.ref_embeddings:
                try:
                    rep = DeepFace.represent(
                        img_path=processed_face,
                        model_name=self.deepface_model,
                        detector_backend="skip",
                        enforce_detection=False
                    )

                    if isinstance(rep, list) and len(rep) > 0 and 'embedding' in rep[0]:
                        emb = np.array(rep[0]['embedding'], dtype=np.float32)
                        norm = np.linalg.norm(emb)
                        if norm > 0:
                            emb = emb / norm
                            logger.info(f"Live embedding norm: {norm:.3f}")

                            sims = []
                            for ref in self.ref_embeddings:
                                sim = np.dot(emb, ref)   # cosine similarity
                                sims.append(sim)
                            if sims:
                                final_score = max(sims)
                                logger.info(f"Live vs refs sims: {[round(s,3) for s in sims]}")
                        else:
                            logger.warning("Live embedding has zero norm")
                except Exception as e:
                    logger.warning(f"Live embedding error: {e}")
                    final_score = 0.0

            self.best_score = max(self.best_score, final_score)
            self.progress_bar.set(final_score)

            color = (0, 255, 0) if final_score > self.system.config.MATCH_THRESHOLD else (0, 0, 255)
            cv2.rectangle(frame, (x, y), (x+w, y+h), color, 2)
            score_text = f"{final_score*100:.1f}%"
            cv2.putText(frame, score_text, (x, max(y-10, 0)), cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2)

            if final_score > self.system.config.MATCH_THRESHOLD:
                self.consecutive_matches += 1
                status_text = f"MATCHING... {self.consecutive_matches}/{self.system.config.REQUIRED_CONSECUTIVE}"
                status_color = "#32CD32"
            else:
                status_text = "NO MATCH"
                status_color = "red"

            self.info_label.configure(
                text=f"Score: {final_score*100:.1f}% | Best: {self.best_score*100:.1f}% | Status: {status_text} | Success: {self.consecutive_matches}/{self.system.config.REQUIRED_CONSECUTIVE}",
                text_color=status_color
            )

            if self.consecutive_matches >= self.system.config.REQUIRED_CONSECUTIVE:
                logger.info(f"Verification successful for {self.best_score}%")
                self.finalize_verification(True)
        else:
            self.info_label.configure(text="No face detected. Please ensure good lighting.", text_color="orange")
            self.progress_bar.set(0)

    def finalize_verification(self, success):
        end_time = time.time()
        runtime = end_time - self.verification_start_time if self.verification_start_time else 0.0
        final_score = self.best_score*100
        timestamp = datetime.now().isoformat()
        student_id = self.current_student_id
        student_name = self.current_student_name
        student_program = self.current_student_program
        self.system.log_verification(student_id, student_name, student_program, success, timestamp, score=final_score, verification_time=runtime)
        self.stop_verification_process()
        try:
            result_logger.save_result(member_name=self.member_name, student_id=student_id, score=float(self.best_score), runtime=runtime,
                success=success)
        except Exception as e:
            print(f"Failed to log result: {e}")
        if success:
            self.status_label.configure(text="ACCESS GRANTED", text_color="green")
            self.info_label.configure(text=f"Welcome, {student_name}! Final Score: {final_score:.1f}%", text_color="green")
            self.tts_system.speak(f"Verification successful. Welcome, {student_name}. Access granted.")
            self.show_student_details_screen(student_id, final_score)
        else:
            self.status_label.configure(text="ACCESS DENIED", text_color="red")
            self.info_label.configure(text=f"Face not recognized. Best score: {final_score:.1f}%", text_color="red")
        self.after(5000, self.reset_labels)

    def reset_labels(self):
        self.status_label.configure(text="Welcome! Click 'Start Verification' to begin.", text_color=("white", "black"))
        self.progress_bar.set(0)
        self.info_label.configure(text="")

    def on_closing(self):
        if self.verification_active:
            self.stop_verification_process()
        if self.cap and self.cap.isOpened():
            self.cap.release()
            self.cap = None
        try:
            if hasattr(self, 'tts_system'):
                self.tts_system.cleanup()
        except Exception as e:
            logger.error(f"Error stopping TTS system: {e}")
        self.quit()
        self.destroy()

    def show_student_details_screen(self, student_id: str, verification_score: float):
        """Display detailed screen with student information and voice announcement"""
        student_info = self.system.get_student_info(student_id)
        if not student_info:
            return
        
        # Create details window
        details_window = Toplevel(self)
        details_window.title("Verification Successful")
        details_window.geometry("600x500")
        details_window.configure(bg='#1a1a1a')
        details_window.resizable(False, False)
        
        # Make window modal and center it
        details_window.transient(self)
        details_window.grab_set()
        
        # Center the window
        details_window.update_idletasks()
        x = (details_window.winfo_screenwidth() // 2) - (300)
        y = (details_window.winfo_screenheight() // 2) - (250)
        details_window.geometry(f"600x500+{x}+{y}")
        
        # Create main frame
        main_frame = ctk.CTkFrame(details_window, fg_color="transparent")
        main_frame.pack(fill="both", expand=True, padx=20, pady=20)
        
        # Success header
        success_label = ctk.CTkLabel(
            main_frame, 
            text="VERIFICATION SUCCESSFUL",
            font=ctk.CTkFont(size=24, weight="bold"),
            text_color="#00ff00"
        )
        success_label.pack(pady=(0, 20))
        
        # Student photo - load from card_photos folder
        photo_frame = ctk.CTkFrame(main_frame, width=180, height=200, fg_color="#2b2b2b")
        photo_frame.pack(pady=(0, 20))
        photo_frame.pack_propagate(False)  # Maintain frame size
        
        # Try to load student photo from card_photos folder
        photo_loaded = False
        try:
            # Look for photo in card_photos folder with student ID as filename
            card_photos_folder = Path("card_photos")
            
            # Try different extensions
            for ext in ['.jpg', '.jpeg', '.png', '.bmp']:
                photo_path = card_photos_folder / f"{student_id}{ext}"
                if photo_path.exists():
                    img = cv2.imread(str(photo_path))
                    if img is not None:
                        # Convert BGR to RGB
                        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                        
                        # Resize image to fit frame (170x190 with padding)
                        img_pil = Image.fromarray(img_rgb)
                        img_pil = img_pil.resize((170, 190), Image.Resampling.LANCZOS)
                        
                        # Create CTk image
                        photo_ctk = ctk.CTkImage(light_image=img_pil,dark_image=img_pil,size=(170, 190))
                        
                        # Display the photo
                        photo_label = ctk.CTkLabel(photo_frame,image=photo_ctk,text="")
                        photo_label.pack(expand=True, padx=5, pady=5)
                        photo_loaded = True
                        logger.info(f"Loaded student photo: {photo_path}")
                        break            
        except Exception as e:
            logger.warning(f"Could not load student photo: {e}")
        
        # If no photo loaded, show placeholder
        if not photo_loaded:
            photo_label = ctk.CTkLabel(
                photo_frame, 
                text="👤\nNo Photo\nAvailable",
                font=ctk.CTkFont(size=14),
                text_color="#cccccc"
            )
            photo_label.pack(expand=True)
            logger.info(f"No photo found for student {student_id} in card_photos folder")
        
        # Student details frame
        details_frame = ctk.CTkFrame(main_frame, fg_color="#2b2b2b")
        details_frame.pack(fill="x", pady=(0, 20))
        
        # Extract student information
        name = student_info.get("name", "N/A")
        program = student_info.get("program", "N/A")
        batch = student_info.get("batch", "N/A")
        academic_performance = student_info.get("academic_performance", "N/A")
        cgpa = student_info.get("cgpa", "N/A")
        
        # Create detail labels
        details_data = [
            ("👤 Name:", name),
            ("📚 Program:", program),
            ("🎓 Batch:", f"20{batch}" if batch.isdigit() else batch),
            ("⭐ Academic Performance:", academic_performance),
            ("📊 CGPA:", cgpa),
            ("Verification Score:", f"{verification_score:.1f}%")
        ]
        
        for i, (label, value) in enumerate(details_data):
            detail_frame = ctk.CTkFrame(details_frame, fg_color="transparent")
            detail_frame.pack(fill="x", padx=20, pady=5)
            
            label_widget = ctk.CTkLabel(
                detail_frame,
                text=label,
                font=ctk.CTkFont(size=14, weight="bold"),
                anchor="w"
            )
            label_widget.pack(side="left", fill="x", expand=True)
            
            value_widget = ctk.CTkLabel(
                detail_frame,
                text=str(value),
                font=ctk.CTkFont(size=14),
                text_color="#00bfff",
                anchor="e"
            )
            value_widget.pack(side="right")
        
        # Buttons frame
        button_frame = ctk.CTkFrame(main_frame, fg_color="transparent")
        button_frame.pack(fill="x", pady=(10, 0))
        
        # Close button
        close_button = ctk.CTkButton(
            button_frame,
            text="✓ Close",
            command=details_window.destroy,
            fg_color="#2196F3",
            hover_color="#1976D2"
        )
        close_button.pack(side="right")
        
        # Auto-close timer
        def auto_close():
            try:
                if details_window.winfo_exists():
                    details_window.destroy()
            except:
                pass
        
        # Auto close after 15 seconds
        details_window.after(15000, auto_close)
        
        # Voice announcement
        self.announce_student_details(name, program, batch, academic_performance)

    def announce_student_details(self, name: str, program: str, batch: str, academic_performance: str):
        """Create and speak the announcement text"""
        # Create a natural announcement
        announcement_parts = [
            f"Verification successful.",
            f"Welcome, {name}.",
            f"Program: {program}.",
            f"Batch 20{batch}." if batch.isdigit() else f"Batch {batch}.",
            f"Academic performance: {academic_performance}.",
            "Access granted."
        ]
        
        announcement_text = " ".join(announcement_parts)
        
        # Log the announcement
        logger.info(f"Voice announcement: {announcement_text}")
        
        # Speak the announcement
        self.tts_system.speak(announcement_text)

# ------------------------
# Main
# ------------------------
if __name__ == "__main__":
    try:
        config = Config()
        backend_system = StudentSystem(config)
        app = App(student_system=backend_system)
        app.mainloop()
    except KeyboardInterrupt:
        print("Interrupted by user")
    except Exception as e:
        logger.critical(f"Critical system error: {e}")
        print(f"Critical error: {e}")


2025-08-31 02:44:52,825 - INFO - Loaded data for 3 students
2025-08-31 02:44:52,827 - INFO - Student Verification System backend initialized
2025-08-31 02:45:27,693 - INFO - Verification logged: None - FAILED (Score: 0.000)


✓ Result saved for Member3: Run #6, Score=0.000, Runtime=16.490s, Status=FAILED


2025-08-31 02:45:46,750 - INFO - Total images loaded for student S003: 30
2025-08-31 02:46:13,728 - INFO - Loaded ref embedding norm: 1.000
2025-08-31 02:46:14,878 - INFO - Loaded ref embedding norm: 1.000
2025-08-31 02:46:16,016 - INFO - Loaded ref embedding norm: 1.000
2025-08-31 02:46:17,080 - INFO - Loaded ref embedding norm: 1.000
2025-08-31 02:46:18,323 - INFO - Loaded ref embedding norm: 1.000
2025-08-31 02:46:19,470 - INFO - Loaded ref embedding norm: 1.000
2025-08-31 02:46:20,419 - INFO - Loaded ref embedding norm: 1.000
2025-08-31 02:46:21,182 - INFO - Loaded ref embedding norm: 1.000
2025-08-31 02:46:22,153 - INFO - Loaded ref embedding norm: 1.000
2025-08-31 02:46:23,026 - INFO - Loaded ref embedding norm: 1.000
2025-08-31 02:46:24,093 - INFO - Loaded ref embedding norm: 1.000
2025-08-31 02:46:25,119 - INFO - Loaded ref embedding norm: 1.000
2025-08-31 02:46:26,219 - INFO - Loaded ref embedding norm: 1.000
2025-08-31 02:46:27,284 - INFO - Loaded ref embedding norm: 1.000
20

✓ Result saved for Member3: Run #7, Score=0.000, Runtime=24.264s, Status=SUCCESS


2025-08-31 02:46:59,709 - INFO - Loaded student photo: card_photos\S003.jpg
2025-08-31 02:46:59,924 - INFO - Imported existing <module 'comtypes.gen' from 'c:\\Users\\User\\AppData\\Local\\Programs\\Python\\Python310\\lib\\site-packages\\comtypes\\gen\\__init__.py'>
2025-08-31 02:46:59,952 - INFO - Using writeable comtypes cache directory: 'c:\Users\User\AppData\Local\Programs\Python\Python310\lib\site-packages\comtypes\gen'
2025-08-31 02:47:03,568 - INFO - Voice announcement: Verification successful. Welcome, Hew. Program: RSW. Batch 2022. Academic performance: First Class. Access granted.
Exception in Tkinter callback
Traceback (most recent call last):
  File "c:\Users\User\AppData\Local\Programs\Python\Python310\lib\tkinter\__init__.py", line 1921, in __call__
    return self.func(*args)
  File "c:\Users\User\AppData\Local\Programs\Python\Python310\lib\tkinter\__init__.py", line 839, in callit
    func(*args)
  File "C:\Users\User\AppData\Local\Temp\ipykernel_23928\3056295522.py", l